# Redes neuronales junto con NeuralODEs

Vamos a mezclar los ejemplos vistos anteriormente, para ello vamos a crear una red neuronal la cual también pase por una NeuralODE, en este caso prefiero poner una aumentada y así evitar el problema anterior. La teoria de estas ya las vimos antes

In [ ]:
using Flux
using DifferentialEquations
using DiffEqFlux
using Plots
using Flux: train!
using Distributions
using DiffEqFlux, DifferentialEquations
using Statistics, LinearAlgebra, Plots
using Flux.Data: DataLoader, Flux
using Optimization, OptimizationOptimJL
using OptimizationFlux, Random

Comenzamos como simpre generando los dastos que vamos a utilizar

In [ ]:
function Noise_Sine(x)
    return sin(2π*x) + rand(Normal(0,0.05))
end;

In [ ]:
t_train = Float32.(hcat(0:0.01:2...))
y_train = Float32.(Noise_Sine.(t_train))
y_test = Float32.(Noise_Sine.(t_train))
trange = t_train[1,:]
tspan = (t_train[1], t_train[end])
t_train

In [ ]:
scatter(t_train[1,:], y_train[1,:], label="training data", title="Sine function with noise")

In [ ]:
dudt = Chain(Dense(21 => 30, celu),
            Dense(30 => 30, relu),
            Dense(30 => 25, relu),
            Dense(25 => 21, tanh_fast)) # Creamos el modelo que va a ser nuestra función diferenciada

A esta NN la vamos a hacer pasar por una NerualODE y extraemos los paráemtros de este modelo

In [ ]:
diffeqarray_to_array(x) = reshape(x, size(x)[1:2]) # Esto acomoda la solución de la EDO en un arreglo de 2 dimensiones 21 x length(trange)

n_ode = NeuralODE(dudt, tspan, Tsit5(), save_everystep = false,
reltol = 1e-3, abstol = 1e-3, save_start = false)
n_ode = AugmentedNDELayer(n_ode, 20)
ps = n_ode.p
model = Chain((x, p = n_ode.p) -> n_ode(x, p), # En primer lugar manda el input a la red neuronal y luego los parámetros
                Array,  # Lo que devuelve la NODE es la solución desde t0 a t1 y devuelve f en cada paso de tiempo
                diffeqarray_to_array, # Esto solo deja la matriz 21x201
                Dense(21, 1)) # Esta f pasa por una capa densa para que la salida sea un número

Creamos la función costo para esta red neruronal, igual que antes usamos mse

In [ ]:
data = Flux.dataloader(t_train, y_train, batchsize = 20, shuffle = true)
loss_node(x, y) = mean((model(x) .- y) .^ 2)

In [ ]:
accuraccy_train = []
accuraccy_test = []
loss_train = []
loss_test = []

In [ ]:
accuracy_train = []
accuracy_test = []
function accuracy(y)
    num_correct = 0
    predictions = model(t_train)
    for i in 1:length(predictions)
        if abs(predictions[1,i] - y[1,i]) < 0.1
            num_correct += 1
        end
    end
    return (num_correct/length(predictions)) * 100.0
end

Seteamos la cantidad de épocas a entrenar, el ratio de aprendizaje y el modelo de optimización que vamos a utilizar. Además creamos la función callback para obtener información de cada época de entrenamiento

In [ ]:
opt = ADAM(0.005)
iter = 0
cb = function()
    global iter
    iter += 1
    if iter % length(data) == 0
        actual_loss = loss_node(data.data[1], data.data[2])
        println("Iteration $iter || Loss = $actual_loss")
        push!(loss_train, actual_loss)
        push!(loss_test, loss_node(data.data[1], y_test))
        push!(accuracy_train, accuracy(y_train))
        push!(accuracy_test, accuracy(y_test))
    end
end

for _ in 1:150
    Flux.train!(loss_node, Flux.params(ps, model), data, opt, cb = cb)
end

In [ ]:
scatter(t_train[1,:],y_train[1,:], label="Train data", title="Predicción de la función seno con ruido")
scatter!(t_train[1,:],y_test[1,:], label="Test data")
scatter!(t_train[1,:],model(t_train)[1,:],label = "predicción")

Ahora veamos el Loss en función de las épocas de entrenamiento

In [ ]:
plot(loss_train, label="train loss",xlabel = "Epochs", ylabel = "Loss", title = "Loss on train vs Epochs")

In [ ]:
plot(loss_test, label="test loss",xlabel = "Epochs", ylabel = "Loss", title = "Loss on test vs Epochs")

In [ ]:
println("Maximum accuracy on train: ", maximum(accuracy_train), "%")
plot(accuracy_train, label="train accuracy",xlabel = "Epochs", ylabel = "Accuracy", title = "Accuracy on train vs Epochs")

In [ ]:
println("Maximum accuracy on test: ", maximum(accuracy_test), "%")
plot(accuracy_test, label = "test accuracy",xlabel = "Epochs", ylabel = "Accuracy", title = "Accuracy on test vs Epochs")